In [1]:
import pandas as pd
import numpy as np
import io
import re

In [7]:
#dataDic = pd.read_csv("DataDictionary.csv", sep = ';')
#dataDic = pd.read_csv("DataDictionaryFromDrawIO.csv", sep = ',')
dataDic = pd.read_csv("DataDictionary_Gruppe2_v4.csv", sep = ',')

In [8]:
## automatically add brackets [] to values with odd characters
def addBrackets(name):
    if type(name) != str:
        return name
    return name if not re.match(r".*[^a-zA-Z0-9_äöüÄÖÜß].*", name) else "["+name+"]"
dataDic.Tabelle = dataDic.Tabelle.map(addBrackets)
dataDic.Feldname = dataDic.Feldname.map(addBrackets)
dataDic.Referenz = dataDic.Referenz.map(addBrackets)

In [9]:
## order the tables based on references
## tables that require foreign keys should be created after the reference table is created
temp_Ref = pd.unique(dataDic.Tabelle).tolist()
out = []
## in case of circle or self references the loop should terminate
i = len(temp_Ref)
while temp_Ref and i > 0:
    out = temp_Ref + out
    temp_Ref = dataDic.loc[[x and y for x,y in zip(dataDic.FK, [x in temp_Ref for x in dataDic.Tabelle])],"Referenz"].to_list()
    i -= 1
tables = pd.unique(np.array(out))

In [10]:
## write to .sql
out = io.open("CreateTableFromDrawIO.sql", "w")
for tableName in tables:
    table = dataDic.loc[dataDic.Tabelle == tableName,:]
    out.write(f"CREATE TABLE {tableName}(\n")
    for i in range(table.shape[0]):
        out.write(f"{table.iloc[i,1]} {table.iloc[i,2]}{f"({int(table.iloc[i,3])})" if not np.isnan(table.iloc[i,3]) else ""}{" NOT NULL" if table.iloc[i,8] else ""},\n")
    foreignKeyTable = table.loc[np.array(table.FK),:]
    for i in range(foreignKeyTable.shape[0]):
        out.write(f"FOREIGN KEY ({foreignKeyTable.iloc[i,1]}) REFERENCES {foreignKeyTable.iloc[i,6]}({foreignKeyTable.iloc[i,1]}),\n")
    primaryKeyTable = table.loc[np.array(table.PK),:]
    out.write("PRIMARY KEY (")
    for i in range(primaryKeyTable.shape[0]):
        out.write(f"{primaryKeyTable.iloc[i,1]} {"AUTOINCREMENT" if table.iloc[i,7] else ""}{", "if i < primaryKeyTable.shape[0]-1 else ")"}")
    out.write("\n);\n\n")
out.close()

In [11]:
for tableName in tables:
    table = dataDic.loc[dataDic.Tabelle == tableName,:]
    print(f"INSERT INTO {tableName}(", end = "")
    for i in range(table.shape[0]):
        print(f"{table.iloc[i,1] if ~ table.iloc[i,7] else ""}{", "if (i < table.shape[0]-1) & ~ table.iloc[i,7] else ""}", end = "")
    print(")\nSELECT DISTINCT\nFROM\n")
    

INSERT INTO Tier_Gattung(Beschreibung)
SELECT DISTINCT
FROM

INSERT INTO Person(Anrede, Titel, Geschlecht, Vorname, Nachname, Straße, Hausnummer, PLZ, Ort, Land, [Telefon Festnetz], [Telefon Mobil], Faxnummer, [E-Mail], Bemerkungen)
SELECT DISTINCT
FROM

INSERT INTO Besucher_Kategorie(Bezeichnung, Beschreibung)
SELECT DISTINCT
FROM

INSERT INTO Tier_Art(Tier_Gattung_ID, Beschreibung)
SELECT DISTINCT
FROM

INSERT INTO Gehege_Kategorie(Bezeichnung)
SELECT DISTINCT
FROM

INSERT INTO Mitarbeiter([Beschäftigung seit], Person_ID, Geburtsdatum, Geburtsort, Nationalität, [soz. Status], Lohnsteuerklasse)
SELECT DISTINCT
FROM

INSERT INTO Futter_Arten(Beschreibung)
SELECT DISTINCT
FROM

INSERT INTO Besucher(Person_ID, Besucher_Kategorie_ID, Bankverbindung)
SELECT DISTINCT
FROM

INSERT INTO Gebäude(Gebäudename, Himmelsrichtung, Beschreibung)
SELECT DISTINCT
FROM

INSERT INTO Lieferant(Person_ID, Firmenname, Vorname_ASP, Nachname_ASP, [für Zoo tätig seit], Bankverbindung, Umsatzsteuernummer)
SELEC

In [12]:
for tableName in tables:
    table = dataDic.loc[dataDic.Tabelle == tableName,:]
    print(f"CREATE TABLE {tableName}(")
    for i in range(table.shape[0]):
        print(f"{table.iloc[i,1]} {table.iloc[i,2]}{f" ({int(table.iloc[i,3])})" if not np.isnan(table.iloc[i,3]) else ""}{" NOT NULL" if table.iloc[i,8] else ""},")
    foreignKeyTable = table.loc[np.array(table.FK),:]
    for i in range(foreignKeyTable.shape[0]):
        print(f"FOREIGN KEY ({foreignKeyTable.iloc[i,1]}) REFERENCES {foreignKeyTable.iloc[i,6]}({foreignKeyTable.iloc[i,1]}),")
    primaryKeyTable = table.loc[np.array(table.PK),:]
    print("PRIMARY KEY (", end = "")
    for i in range(primaryKeyTable.shape[0]):
        print(f"{primaryKeyTable.iloc[i,1]} {"AUTOINCREMENT" if table.iloc[i,7] else ""}{", "if i < primaryKeyTable.shape[0]-1 else ")"}", end = "")
    print("\n);\n")

    
## CREATE TABLE XXTabelle1 (
## XXFeldnameN XXFelddatentypeN XXLaenge_ZeichenN XXNNN,
## ....
## FOREIGN KEY (XXFeldnameN) REFERENCES XXReferenz(XXFeldnameN),
## ....
## PRIMARY KEY (XXFeldnameN XXAI, ....)

CREATE TABLE Tier_Gattung(
Tier_Gattung_ID INTEGER NOT NULL,
Beschreibung TEXT (50),
PRIMARY KEY (Tier_Gattung_ID AUTOINCREMENT)
);

CREATE TABLE Person(
Person_ID INTEGER NOT NULL,
Anrede TEXT (4),
Titel TEXT (10),
Geschlecht TEXT (10),
Vorname TEXT (50),
Nachname TEXT (50),
Straße TEXT (10),
Hausnummer TEXT (50),
PLZ TEXT (10),
Ort TEXT (50),
Land TEXT (50),
[Telefon Festnetz] TEXT (15),
[Telefon Mobil] TEXT (15),
Faxnummer TEXT (50),
[E-Mail] TEXT (64),
Bemerkungen TEXT (255),
PRIMARY KEY (Person_ID AUTOINCREMENT)
);

CREATE TABLE Besucher_Kategorie(
Besucher_Kategorie_ID INTEGER NOT NULL,
Bezeichnung TEXT (50),
Beschreibung TEXT (50),
PRIMARY KEY (Besucher_Kategorie_ID AUTOINCREMENT)
);

CREATE TABLE Tier_Art(
Tier_Art_ID INTEGER NOT NULL,
Tier_Gattung_ID INTEGER NOT NULL,
Beschreibung TEXT (255),
FOREIGN KEY (Tier_Gattung_ID) REFERENCES Tier_Gattung(Tier_Gattung_ID),
PRIMARY KEY (Tier_Art_ID AUTOINCREMENT)
);

CREATE TABLE Gehege_Kategorie(
Gehege_Kategorie_ID INTEGER NOT NULL,
Be